In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# Исследование NPS телекоммуникационной компании среди клиентов России

- [Описание проекта](#project_description)
- [Выгрузка данных](#data_upload)
- [Дашборды на сайте Tableau Public](#dashboards)
- [pdf-файл с презентацией](#presentation)

## Описание проекта <a name='project_description'></a>

Используя дашборд, нужно ответить на вопросы:

* Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?

* Какие группы пользователей наиболее лояльны к сервису? Какие менее?

* Какой общий NPS среди всех опрошенных?

* Как можно описать клиентов, которые относятся к группе cторонников?

## Выгрузка данных <a name='data_upload'></a>

In [2]:
path_to_db = '../datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query = """
        SELECT
            u.user_id,
            u.lt_day,
            CASE 
                WHEN u.lt_day <= 365 THEN 1 
                ELSE 0 
            END AS is_new,
            u.age,
            CASE 
                WHEN u.gender_segment = 1 THEN 'женщина' 
                WHEN u.gender_segment = 0 THEN 'мужчина' 
                ELSE u.gender_segment
            END AS gender_segment,
            u.os_name,
            u.cpe_type_name,
            l.country,
            l.city, 
            CASE 
                WHEN SUBSTR(a_s.title, INSTR(a_s.title, ' ') + 1) = 'n/a' THEN Null
                ELSE SUBSTR(a_s.title, INSTR(a_s.title, ' ') + 1) 
            END AS age_segment, 
            SUBSTR(t_s.title, INSTR(t_s.title, ' ') + 1) AS traffic_segment,
            SUBSTR(l_s.title, INSTR(l_s.title, ' ') + 1) AS lifetime_segment,
            u.nps_score,
            CASE
                WHEN u.nps_score <= 6 THEN 'критик'
                WHEN u.nps_score <= 8 THEN 'нейтрал'
                ELSE 'сторонник'
            END AS nps_group
        FROM user AS u
        JOIN location AS l USING (location_id)
        JOIN age_segment AS a_s USING (age_gr_id)
        JOIN traffic_segment AS t_s USING (tr_gr_id)
        JOIN lifetime_segment AS l_s USING (lt_gr_id)
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(5)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонник
1,A001WF,2344,0,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонник
2,A003Q7,467,0,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонник
3,A004TB,4190,0,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,сторонник
4,A004XT,1163,0,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,сторонник


In [5]:
df.to_csv('../datasets/telecomm_csi_tableau.csv', index=False)

## Дашборды на сайте Tableau Public <a name='dashboards'></a>

https://public.tableau.com/views/2_16516078441580/1_?:language=en-US&:display_count=n&:origin=viz_share_link

## pdf-файл с презентацией <a name='presentation'></a>

https://drive.google.com/file/d/1c86VfoLOJlqboG-Vrnw_63WwdsH-iCrA/view?usp=sharing